# 03: Polygon Triangulation

*Authors: Jan Erik Swiadek, Prof. Dr. Kevin Buchin*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases implementations of algorithms and data structures presented in the lecture, and it elaborates on some practical considerations concerning their use.
Furthermore, it provides interactive visualisations and animations.

## Table of Contents

1. Introduction  
2. Setup  
3. Algorithms  
    3.1. Recursive Triangulation  
    3.2. Monotone Partitioning  
    3.3. Piecewise Triangulation  
4. Art Gallery Problem  
5. References  

## 1. Introduction

A **polygon triangulation** is ...

## 2. Setup

First let's do some setup. This is not very interesting, so you can skip to Section 3 if you want.

We now import everything we'll need throughout this notebook from external modules, including our module for generic data structures, our module for common geometric primitives and operations as well as our module for visualisation purposes. (The data structure and geometry modules will probably receive their own notebook later.)

In [ ]:
from typing import Any, Callable, Optional
from enum import Enum, auto
import copy

# Data structure, geometry and visualisation module imports
from modules.data_structures import BinaryTreeDict, Comparator, ComparisonResult as CR
from modules.geometry import Point, LineSegment, DoublyConnectedSimplePolygon, Vertex, HalfEdge, Orientation as ORT
from modules.visualisation import VisualisationTool, SimplePolygonInstance, LineSegmentsMode

Additionally, we create an object for our visualisation tool and register a few example instances.

In [ ]:
visualisation_tool = VisualisationTool(400, 400, SimplePolygonInstance())   # TODO: Probably should change PolygonMode behaviour...

square = DoublyConnectedSimplePolygon()
square.add_vertex(Point(150, 150))
square.add_vertex(Point(150, 250))
square.add_vertex(Point(250, 250))
square.add_vertex(Point(250, 150))
visualisation_tool.register_example_instance("square", square)

mouse = DoublyConnectedSimplePolygon()
mouse.add_vertex(Point(116.75, 176))
mouse.add_vertex(Point(43.75, 116))
mouse.add_vertex(Point(46.75, 51))
mouse.add_vertex(Point(102.75, 23))
mouse.add_vertex(Point(135.75, 53))
mouse.add_vertex(Point(191.75, 73))
mouse.add_vertex(Point(211.75, 34))
mouse.add_vertex(Point(289.75, 13))
mouse.add_vertex(Point(354.75, 50))
mouse.add_vertex(Point(308.75, 74))
mouse.add_vertex(Point(253.75, 97))
mouse.add_vertex(Point(237.75, 124))
mouse.add_vertex(Point(299.75, 145))
mouse.add_vertex(Point(370.75, 182))
mouse.add_vertex(Point(379.75, 265))
mouse.add_vertex(Point(328.75, 291))
mouse.add_vertex(Point(280.75, 270))
mouse.add_vertex(Point(264.75, 229))
mouse.add_vertex(Point(233.75, 186))
mouse.add_vertex(Point(193.75, 193))
mouse.add_vertex(Point(186.75, 238))
mouse.add_vertex(Point(177.75, 286))
mouse.add_vertex(Point(148.75, 306))
mouse.add_vertex(Point(92.75, 285))
mouse.add_vertex(Point(82.75, 236))
visualisation_tool.register_example_instance("mouse", mouse)

wicked = DoublyConnectedSimplePolygon()
wicked.add_vertex(Point(58.75, 308))
wicked.add_vertex(Point(43.75, 356))
wicked.add_vertex(Point(73.75, 388))
wicked.add_vertex(Point(21.75, 385))
wicked.add_vertex(Point(10.75, 359))
wicked.add_vertex(Point(19.75, 322))
wicked.add_vertex(Point(33.75, 259))
wicked.add_vertex(Point(39.75, 229))
wicked.add_vertex(Point(106.75, 244))
wicked.add_vertex(Point(175.75, 236))
wicked.add_vertex(Point(167.75, 207))
wicked.add_vertex(Point(112.75, 208))
wicked.add_vertex(Point(70.75, 193))
wicked.add_vertex(Point(27.75, 167))
wicked.add_vertex(Point(57.75, 112))
wicked.add_vertex(Point(9.75, 78))
wicked.add_vertex(Point(25.75, 26))
wicked.add_vertex(Point(34.75, 71))
wicked.add_vertex(Point(100.75, 59))
wicked.add_vertex(Point(61.75, 30))
wicked.add_vertex(Point(102.75, 7))
wicked.add_vertex(Point(154.75, 18))
wicked.add_vertex(Point(126.75, 30))
wicked.add_vertex(Point(177.75, 64))
wicked.add_vertex(Point(231.75, 99))
wicked.add_vertex(Point(235.75, 71))
wicked.add_vertex(Point(212.75, 55))
wicked.add_vertex(Point(232.75, 22))
wicked.add_vertex(Point(304.75, 17))
wicked.add_vertex(Point(362.75, 39))
wicked.add_vertex(Point(360.75, 76))
wicked.add_vertex(Point(339.75, 80))
wicked.add_vertex(Point(277.75, 85))
wicked.add_vertex(Point(264.75, 111))
wicked.add_vertex(Point(247.75, 179))
wicked.add_vertex(Point(235.75, 129))
wicked.add_vertex(Point(195.75, 98))
wicked.add_vertex(Point(125.75, 115))
wicked.add_vertex(Point(107.75, 151))
wicked.add_vertex(Point(124.75, 181))
wicked.add_vertex(Point(173.75, 146))
wicked.add_vertex(Point(210.75, 174))
wicked.add_vertex(Point(234.75, 224))
wicked.add_vertex(Point(267.75, 234))
wicked.add_vertex(Point(282.75, 212))
wicked.add_vertex(Point(278.75, 167))
wicked.add_vertex(Point(307.75, 120))
wicked.add_vertex(Point(382.75, 130))
wicked.add_vertex(Point(377.75, 221))
wicked.add_vertex(Point(349.75, 185))
wicked.add_vertex(Point(335.75, 233))
wicked.add_vertex(Point(380.75, 301))
wicked.add_vertex(Point(359.75, 351))
wicked.add_vertex(Point(327.75, 329))
wicked.add_vertex(Point(297.75, 273))
wicked.add_vertex(Point(213.75, 274))
wicked.add_vertex(Point(228.75, 326))
wicked.add_vertex(Point(295.75, 300))
wicked.add_vertex(Point(288.75, 353))
wicked.add_vertex(Point(227.75, 362))
wicked.add_vertex(Point(174.75, 342))
wicked.add_vertex(Point(136.75, 272))
wicked.add_vertex(Point(118.75, 353))
wicked.add_vertex(Point(93.75, 308))
wicked.add_vertex(Point(71.75, 347))
visualisation_tool.register_example_instance("wicked", wicked)

irreg = DoublyConnectedSimplePolygon()
irreg.add_vertex(Point(150, 200))
irreg.add_vertex(Point(200, 200))
irreg.add_vertex(Point(250, 200))
visualisation_tool.register_example_instance("irreg", irreg)

## 3. Algorithms

...

### 3.1. Recursive Triangulation

Explanations: Algorithm from proof (see also [1, Theorem 3.1]), diagonal connection via edges and area coordinates for triangle tests (containment check is possible with orientation test, but we need the contained vertex that is "farthest") ...

Note that, currently, the algorithm inputs are copied if used via the visualisation tool. Maybe this should be done in the algorithms themselves to make it more transparent?

In [ ]:
def recursive_pt(polygon: DoublyConnectedSimplePolygon) -> DoublyConnectedSimplePolygon:
    if not polygon.is_simple():
        raise ValueError("Input polygon is not simple.")

    representative_edges = [polygon.topmost_vertex.edge]
    while representative_edges:
        leftmost_edge = _get_leftmost_edge(representative_edges.pop())
        connection_edges = _get_connection_edges(leftmost_edge)
        if connection_edges is not None:
            diagonal = polygon.add_diagonal(connection_edges[0], connection_edges[1])
            representative_edges.extend((diagonal, diagonal.twin))
    return polygon

def _get_leftmost_edge(representative_edge: HalfEdge) -> HalfEdge:
    leftmost_edge, leftmost_vertex = representative_edge, representative_edge.origin
    edge = representative_edge.next
    while edge is not representative_edge:
        vertex = edge.origin
        if vertex.x < leftmost_vertex.x or (vertex.x == leftmost_vertex.x and vertex.y < leftmost_vertex.y):
            leftmost_edge, leftmost_vertex = edge, vertex
        edge = edge.next
    return leftmost_edge

def _get_connection_edges(leftmost_edge: HalfEdge) -> Optional[tuple[HalfEdge, HalfEdge]]:
    edge = leftmost_edge.next.next
    if edge is leftmost_edge.prev:
        return None

    triangle = (leftmost_edge.prev.origin, leftmost_edge.origin, leftmost_edge.next.origin)
    connection_edge, max_coordinate = None, 0.0
    while edge is not leftmost_edge.prev:
        area_coordinates = _get_area_coordinates(edge.origin, triangle)
        if all(0.0 <= coordinate <= 1.0 for coordinate in area_coordinates):
            if connection_edge is None or area_coordinates[1] > max_coordinate:
                connection_edge, max_coordinate = edge, area_coordinates[1]
        edge = edge.next

    if connection_edge is None:
        return leftmost_edge.prev, leftmost_edge.next

    return leftmost_edge, connection_edge
    

def _get_area_coordinates(vertex: Vertex, triangle: tuple[Vertex, Vertex, Vertex]) -> tuple[float, float, float]:
    parallelogram_area = _signed_area(triangle[0], triangle[1], triangle[2])
    return (
        _signed_area(vertex, triangle[1], triangle[2]) / parallelogram_area,
        _signed_area(triangle[0], vertex, triangle[2]) / parallelogram_area,
        _signed_area(triangle[0], triangle[1], vertex) / parallelogram_area
    )

def _signed_area(u: Vertex, v: Vertex, w: Vertex) -> float:
    p, q, r = u.point, v.point, w.point
    return (q - p).cross(r - p)

Runs in $O(n^2)$.

Now visualisation.

In [ ]:
visualisation_tool.register_algorithm("Rec", recursive_pt, LineSegmentsMode(endpoint_radius = 0.0))
# TODO: Implement appropriate mode. Maybe make Triangulation object.

If you haven't used our interactive visualisation tool before, see the convex hull notebook for an explanation. ...

In [ ]:
visualisation_tool.display()

### 3.2. Monotone Partitioning

Partition polygon into $y$-monotone subpolygons using a plane sweep algorithm (cf. LSI notebook). ...

Comparators again.
The event queue comparator is the same as usual.

In [ ]:
# TODO: Do we need EPSILON for robustness?
class EventQueueComparator(Comparator[Vertex]):
    def compare(self, item: Any, key: Vertex) -> CR:
        if not isinstance(item, Vertex):
            raise TypeError("Only vertices can be compared with event vertices.")
        elif item.point == key.point:
            return CR.MATCH
        elif item.y > key.y or (item.y == key.y and item.x < key.x):
            return CR.BEFORE
        else:
            return CR.AFTER

The status structure comparator is similar to the one from the LSI notebook, but simpler since there are no intersections or overlaps.
Furthermore, horizontal edges in the status structure should be deleted in the next step and thus don't make much of a problem... I hope.
Also, the status structure shouldn't ever contain two adjacent edges... I think.
Overall, the comparator doesn't (seem to) need a dynamic dependency on the current event point.

In [ ]:
# TODO: Do we need EPSILON for robustness?
class StatusStructureComparator(Comparator[HalfEdge]):
    def compare(self, item: Any, key: HalfEdge) -> CR:
        if isinstance(item, Vertex):
            return self._compare_vertex_with_edge(item, key)
        elif isinstance(item, HalfEdge):
            return self._compare_edge_with_edge(item, key)
        else:
            raise TypeError("Only vertices and edges can be compared with status edges.")

    def _compare_vertex_with_edge(self, vertex: Vertex, edge: HalfEdge) -> CR:  # This should suffice in all possible cases.
        upper, lower = edge.upper_and_lower
        ort = vertex.point.orientation(lower.point, upper.point)
        if ort is ORT.LEFT:
            return CR.BEFORE
        elif ort is ORT.RIGHT:
            return CR.AFTER
        else:
            return CR.MATCH

    def _compare_edge_with_edge(self, edge1: HalfEdge, edge2: HalfEdge) -> CR:    # This should suffice in all possible cases.
        upper1, lower1 = edge1.upper_and_lower
        upper2, lower2 = edge2.upper_and_lower
        if upper1.y <= upper2.y:
            return self._compare_vertex_with_edge(upper1, edge2)
        else:
            cr = self._compare_vertex_with_edge(upper2, edge1)
            if cr is CR.BEFORE:
                return CR.AFTER
            elif cr is CR.AFTER:
                return CR.BEFORE
            else:
                return CR.MATCH

Enum for the different vertex types and class for helper vertices.

In [ ]:
class VertexType(Enum):
    START = auto()
    END = auto()
    SPLIT = auto()
    MERGE = auto()
    REGULAR = auto()

VT = VertexType

class VertexInfo:
    event_queue_comparator = EventQueueComparator()

    def __init__(self, vertex: Vertex):
        self.incoming_edge: HalfEdge = vertex.edge.prev
        self.outgoing_edge: HalfEdge = vertex.edge

        prev_neighbour = self.incoming_edge.origin
        next_neighbour = self.outgoing_edge.destination
        prev_cr = self.event_queue_comparator.compare(prev_neighbour, vertex)
        next_cr = self.event_queue_comparator.compare(next_neighbour, vertex)
        if prev_cr is CR.AFTER and next_cr is CR.AFTER:
            if vertex.point.orientation(prev_neighbour.point, next_neighbour.point) is ORT.RIGHT:
                self.vertex_type = VT.START
            else:
                self.vertex_type = VT.SPLIT
        elif prev_cr is CR.BEFORE and next_cr is CR.BEFORE:
            if vertex.point.orientation(prev_neighbour.point, next_neighbour.point) is ORT.RIGHT:
                self.vertex_type = VT.END
            else:
                self.vertex_type = VT.MERGE
        else:
            self.vertex_type = VT.REGULAR

Determine vertex types and handle vertices accordingly...

In [ ]:
class Monotoniser:
    def __init__(self, polygon: DoublyConnectedSimplePolygon):
        if not polygon.is_simple():
            raise ValueError("Input polygon is not simple.")
        self._polygon = polygon

        self._status_structure: BinaryTreeDict[HalfEdge, VertexInfo] = BinaryTreeDict(StatusStructureComparator())

        self._event_queue: BinaryTreeDict[Vertex, VertexInfo] = BinaryTreeDict(VertexInfo.event_queue_comparator)
        for vertex in self._polygon._vertices():    # How is this supposed to take O(n) time?
            self._event_queue.insert(vertex, VertexInfo(vertex))

        self.representative_edges: list[HalfEdge] = []

    def make_monotone(self) -> DoublyConnectedSimplePolygon:
        while not self._event_queue.is_empty():
            event_vertex, event_vertex_info = self._event_queue.pop_first()
            handler_name = f"_handle_{str.lower(event_vertex_info.vertex_type.name)}_vertex"
            handler: Callable[[Vertex, VertexInfo], None] = getattr(self, handler_name)
            handler(event_vertex, event_vertex_info)

        return self._polygon

    def _handle_start_vertex(self, _: Vertex, event_vertex_info: VertexInfo):
        self._status_structure.insert(event_vertex_info.outgoing_edge, event_vertex_info)

    def _handle_end_vertex(self, _: Vertex, event_vertex_info: VertexInfo):
        _, helper_info = self._status_structure.delete(event_vertex_info.incoming_edge)
        if helper_info.vertex_type is VT.MERGE:
            diagonal = self._polygon.add_diagonal(event_vertex_info.outgoing_edge, helper_info.outgoing_edge)  # No update should be necessary.
            self.representative_edges.append(diagonal)             # But it should be an anchor edge.
        self.representative_edges.append(event_vertex_info.outgoing_edge)      # And the outgoing edge should always be an anchor edge.

    def _handle_split_vertex(self, event_vertex: Vertex, event_vertex_info: VertexInfo):
        edge_left_of_vertex, helper_info = self._status_structure.search_predecessor(event_vertex)
        diagonal = self._polygon.add_diagonal(event_vertex_info.outgoing_edge, helper_info.outgoing_edge)
        copied_event_vertex_info = copy.copy(event_vertex_info)     # This copy and update combination should suffice.
        copied_event_vertex_info.outgoing_edge = diagonal
        self._status_structure.update(edge_left_of_vertex, lambda _: copied_event_vertex_info)
        self._status_structure.insert(event_vertex_info.outgoing_edge, event_vertex_info)

    def _handle_merge_vertex(self, event_vertex: Vertex, event_vertex_info: VertexInfo):
        _, helper_info = self._status_structure.delete(event_vertex_info.incoming_edge)
        if helper_info.vertex_type is VT.MERGE:
            diagonal = self._polygon.add_diagonal(event_vertex_info.outgoing_edge, helper_info.outgoing_edge)  # No update should be necessary.
            self.representative_edges.append(diagonal)             # But it should be an anchor edge.
        edge_left_of_vertex, helper_info = self._status_structure.search_predecessor(event_vertex)
        if helper_info.vertex_type is VT.MERGE:
            diagonal = self._polygon.add_diagonal(event_vertex_info.outgoing_edge, helper_info.outgoing_edge)
            event_vertex_info.outgoing_edge = diagonal      # This update should suffice.
            self.representative_edges.append(diagonal.twin)    # And the twin should be an anchor edge.
        self._status_structure.update(edge_left_of_vertex, lambda _: event_vertex_info)

    def _handle_regular_vertex(self, event_vertex: Vertex, event_vertex_info: VertexInfo):
        is_interior_to_the_right, helper_info = self._status_structure.delete(event_vertex_info.incoming_edge)
        if is_interior_to_the_right:
            if helper_info.vertex_type is VT.MERGE:
                diagonal = self._polygon.add_diagonal(event_vertex_info.outgoing_edge, helper_info.outgoing_edge)    # No update should be necessary.
                self.representative_edges.append(diagonal)             # But it should be an anchor edge.
            self._status_structure.insert(event_vertex_info.outgoing_edge, event_vertex_info)
        else:
            edge_left_of_vertex, helper_info = self._status_structure.search_predecessor(event_vertex)
            if helper_info.vertex_type is VT.MERGE:
                diagonal = self._polygon.add_diagonal(event_vertex_info.outgoing_edge, helper_info.outgoing_edge)
                event_vertex_info.outgoing_edge = diagonal   # This update should suffice.
                self.representative_edges.append(diagonal.twin)    # And the twin should be an anchor edge.
            self._status_structure.update(edge_left_of_vertex, lambda _: event_vertex_info)

Let's register this as a standalone algorithm.

In [ ]:
visualisation_tool.register_algorithm("Mon", lambda x: Monotoniser(x).make_monotone(), LineSegmentsMode(endpoint_radius = 0.0))
# TODO: Implement appropriate mode. Maybe make Triangulation object.

Visualisation incoming.

In [ ]:
visualisation_tool.display()

### 3.3. Piecewise Triangulation

Nice description from [2]:
"The idea behind the triangulation algorithm is quite simple:
Try to triangulate *everything* you can [above] the current vertex by adding diagonals, and then remove the triangulated region from further consideration.
The trickiest aspect of implementing this idea is finding a clean invariant that characterizes the *untriangulated region* that lies [above the current vertex]." ...

In [ ]:
def triangulate_polygon(polygon: DoublyConnectedSimplePolygon) -> DoublyConnectedSimplePolygon:
    monotoniser = Monotoniser(polygon)
    polygon = monotoniser.make_monotone()

    for anchor_edge in monotoniser.representative_edges:
        topmost_edge = anchor_edge          # unify style with recursive algorithm.
        edge = anchor_edge.next
        while edge is not anchor_edge:
            if VertexInfo.event_queue_comparator.compare(edge.origin, topmost_edge.origin) is CR.BEFORE:
                topmost_edge = edge
            edge = edge.next

        edges: list[tuple[HalfEdge, bool]] = []      # TODO: Use namedtuple?
        left_chain_edge = topmost_edge
        right_chain_edge = topmost_edge.prev
        cr = VertexInfo.event_queue_comparator.compare(left_chain_edge.origin, right_chain_edge.origin)
        while True:
            if cr is CR.BEFORE:
                edges.append((left_chain_edge, True))
                left_chain_edge = left_chain_edge.next
            elif cr is CR.AFTER:
                edges.append((right_chain_edge, False))
                right_chain_edge = right_chain_edge.prev
            else:
                edges.append((left_chain_edge, True))
                break
            cr = VertexInfo.event_queue_comparator.compare(left_chain_edge.origin, right_chain_edge.origin)

        stack = edges[0:2]
        for edge in edges[2:-1]:
            if edge[1] != stack[-1][1]:
                prev_edge = stack[-1]
                while len(stack) >= 2:
                    polygon.add_diagonal(edge[0], stack.pop()[0])
                stack.pop()
                stack.extend((prev_edge, edge))
            else:
                ort = ORT.LEFT if edge[1] else ORT.RIGHT
                popped_edge = stack.pop()
                while stack and popped_edge[0].origin.point.orientation(edge[0].origin.point, stack[-1][0].origin.point) is ort:
                    popped_edge = stack.pop()
                    polygon.add_diagonal(edge[0], popped_edge[0])
                stack.append(popped_edge)
                stack.append(edge)

        for stack_edge in stack[1:-1]:
            polygon.add_diagonal(edges[-1][0], stack_edge[0])
    
    return polygon

Register.

In [ ]:
visualisation_tool.register_algorithm("Mon + Tri", triangulate_polygon, LineSegmentsMode(endpoint_radius = 0.0))
# TODO: Implement appropriate mode. Maybe make Triangulation object.

Visualise.

In [ ]:
visualisation_tool.display()

## 4. Art Gallery Problem

Maybe use the simple approach from [3] to solve the *Art Gallery Problem* ... the more illustrative dual graph approach is explained in [1] but its implementation is more involved.

In [ ]:
from modules.geometry import Polygon

def art_gallery(polygon: DoublyConnectedSimplePolygon, triangulation_alg) -> Polygon:    # TODO: Add a PointSet object (or rename Intersections to PointDict).    
    polygon = triangulation_alg(polygon)

    vertex1 = polygon.topmost_vertex
    vertex2 = vertex1.edge.destination
    coloured_vertices = ([vertex1], [vertex2], [])
    prev_prev_colour, prev_colour = 0, 1

    vertex = vertex2
    while vertex is not vertex1:
        is_vertex_degree_odd = True
        incident_edge = vertex.edge.prev.twin
        while incident_edge is not vertex.edge:
            is_vertex_degree_odd = not is_vertex_degree_odd
            incident_edge = incident_edge.prev.twin
        
        vertex = vertex.edge.destination
        if vertex is vertex1:
            break

        vertex_colour = prev_prev_colour if is_vertex_degree_odd else 3 - prev_prev_colour - prev_colour
        coloured_vertices[vertex_colour].append(vertex)
        prev_prev_colour, prev_colour = prev_colour, vertex_colour

    positions = min(coloured_vertices, key = lambda vertices: len(vertices))

    return Polygon(positions)


Register.

In [ ]:
from modules.visualisation import PointsMode

visualisation_tool.register_algorithm("Art Gallery (Rec)", lambda x: art_gallery(x, recursive_pt), PointsMode())
visualisation_tool.register_algorithm("Art Gallery (Mon)", lambda x: art_gallery(x, triangulate_polygon), PointsMode())

Visualise.

In [ ]:
visualisation_tool.display()

## 5. References

[1] Mark de Berg, Otfried Cheong, Marc van Kreveld, and Mark Overmars. *Computational Geometry: Algorithms and Applications*, 3rd Edition, 2008.

[2] David M. Mount. [*CMSC 754: Computational Geometry*](https://www.cs.umd.edu/class/spring2020/cmsc754/Lects/cmsc754-spring2020-lects.pdf), 2020.

[3] Ali A. Kooshesh, and Bernard M. E. Moret. *Three-coloring the Vertices of a Triangulated Simple Polygon*, Pattern Recognition 25(4), p. 443, 1992.